In [ ]:
import os
from typing import overload
import numpy as np
from pymbar import MBAR
import pandas as pd

In [ ]:
round_NO = 1
os.chdir(f'./complex_{round_NO}_10_100_100/post_processing')

In [ ]:
#beta = 1.0 / (300 * 1.380649 * 6.02214076 / 1000)
beta = 1.0 / (300 * 1.380649e-23 )
name_list = [ 0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.35, 0.5, 0.75, 1.0]
#name_list = [ 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.85, 1]

# number of states
K = len(name_list)

mian_dir = os.getcwd()

In [ ]:
os.chdir(mian_dir + '/' + str(name_list[0]))
result_path = os.getcwd() + '/MBAR_energy.txt'

In [ ]:
u_kn = np.loadtxt(result_path)

In [ ]:
u_kn.shape

In [ ]:
for file in range(1,len(name_list)):

    os.chdir(mian_dir + '/' + str(name_list[file]))
    result_path = os.getcwd() + '/MBAR_energy.txt'

    a = np.loadtxt(result_path)
    #print(a.shape)
    #print(a)
    u_kn = np.vstack((u_kn,a))

In [ ]:
u_kn.shape

In [ ]:
N_k = np.full(K,u_kn.shape[1]/K, dtype=np.int32)
N_k

In [ ]:
mbar = MBAR(u_kn,N_k)

In [ ]:
Delta_f_ij, dDelta_f_ij, Theta_ij = mbar.getFreeEnergyDifferences(return_theta=True)

# kbT  -->   kcal/mol   (  *  0.596  )

In [ ]:
dDelta_f_ij

In [ ]:
print('Delta_f_ij.shape:',Delta_f_ij.shape)
print(f"Restraint energy : {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol")
with open(f'{mian_dir}/Restraint_energy.txt','w') as f:
    f.write(f'Restraint energy(MBAR): {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol\n')
    for i in range(K-1):
        f.writelines(f'{Delta_f_ij[i][i+1] * 0.596:.3f}\n')

In [ ]:
pd.DataFrame(Delta_f_ij * 0.596)

In [ ]:
O_ij = mbar.computeOverlap()['matrix']
np.savetxt('../restraint_overlap.txt',O_ij,fmt='%.8f')
O_ij.round(decimals=3, out=O_ij)
pd.DataFrame(O_ij)

In [ ]:
dDelta_f_ij